# 类型系统 

In [1]:
using JupyterFormatter
enable_autoformat()

2-element Vector{Function}:
 revise (generic function with 3 methods)
 format_current_cell (generic function with 1 method)

+ 与二进制存储结构相关联的成为元类型 `primitive type`, 如`Int32`, `Int64`这种，在定义元类型时要声明**位数**， 且位数必须为8的倍数

## 元类型

In [2]:
# 256位正数
primitive type Int256 <: Integer 256 end

In [3]:
# 8位浮点数
primitive type Float8 <: AbstractFloat 8 end

+ 与抽象概念相关，被用来继承的类型为`Abstract Type`, 如`Real`, `Integer`, `AbstractFloat`

## 抽象类型

In [4]:
abstract type name end
abstract type family_name <: name end

In [5]:
family_name <: name

true

In [6]:
supertype(family_name)

name

In [7]:
subtypes(name)

1-element Vector{Any}:
 family_name

In [8]:
# 并不会递归查询
subtypes(Integer)

4-element Vector{Any}:
 Bool
 Int256
 Signed
 Unsigned

In [9]:
subtypes(Signed)

6-element Vector{Any}:
 BigInt
 Int128
 Int16
 Int32
 Int64
 Int8

##  类型的类型

In [10]:
typeof(Int64)

DataType

In [11]:
typeof(AbstractFloat)

DataType

In [12]:
typeof(DataType)

DataType

所有类型的类型是`DataType`，所有类型的父类型是`Any`, 所有类型的子类型都包含`Union`

In [13]:
Real::Any

Real

In [14]:
# DataType和Any都是类型，Any是所有类型的父类型，而DataType是所有类型的类型

DataType::Any

Any::DataType

Any

In [15]:
Union::DataType

Union

In [16]:
# ?
supertype(Union)

Type{T}

`DataType`的父类型是参数化类型`Type{T}`

In [17]:
supertype(DataType)

Type{T}

In [18]:
supertype(Type)

Any

`Type{T}`可以看作类型生成器

In [19]:
isa(Int64, Type{Int64})

true

In [20]:
isa(Pointy, Type{Pointy})

LoadError: UndefVarError: Pointy not defined

##  复合类型

In [21]:
struct FooA
    a::Any
    b::Float64
end

In [22]:
mutable struct FooC
    a::Any
    b::Float64
end

In [23]:
dump(FooC)

FooC <: Any
  a::Any
  b::Float64


In [24]:
fieldnames(FooC)

(:a, :b)

In [25]:
typeof(FooA)

DataType

In [26]:
typeof(FooA(1, 1.2::Float64))

FooA

###  构造函数

+ 按照struct中类型指定，手动指定每一个field的类型
+ 不指定每一个field的类型，julia会自动进行convert到struct的类型指定（不建议）

> 一个struct实例化后，如果更改某一个field的值，他会自动将该值convert到struct的类型指定。

任何一个struct在定义时对field进行了类型声明，那么任何一个实例化的struct的field的值都是该field的类型

In [27]:
fooc = FooC("xue", 23::Int64)

FooC("xue", 23.0)

In [28]:
fooc.b = Int16(2)

2

In [29]:
typeof(fooc.b)

Float64

In [30]:
fooc.b = 2+2im

LoadError: InexactError: Float64(2 + 2im)

In [31]:
mutable struct FooD
    c::Any
    d::FooC
end

In [32]:
food = FooD(1, FooC(2, 3))

FooD(1, FooC(2, 3.0))

In [33]:
food.d.b

3.0

struct的mutable的标识符只控制着本层的可变性，它不会涉及到外层和内层，也不会在上下层中传播

In [34]:
T = TypeVar(:a, Int64, Real)

Int64<:a<:Real

In [35]:
T.name

:a

In [36]:
dump(T)

TypeVar
  name: Symbol a
  lb: Int64 <: Signed
  ub: Real <: Number


## 参数化类型

### 参数化复合类型

In [37]:
mutable struct Point1D{T}
    x::T
end

In [38]:
# Poin1D可以被看作泛型
Point1D{Int64} <: Point1D

true

In [39]:
# 即使参数存在继承关系, 参数化符合类型也不存在继承关系
Point1D{Int64} <: Point1D{Integer}

false

In [40]:
point1 = Point1D{Int64}(1)

Point1D{Int64}(1)

In [41]:
point1.x

1

In [42]:
mutable struct Point3D{T1,T2}
    x::T1
    y::T1
    z::T2
end

In [43]:
point2 = Point3D{Int64,Float64}(1, 1, 2)

Point3D{Int64, Float64}(1, 1, 2.0)

In [44]:
point2.x

1

In [45]:
point2.z

2.0

实例化时，尽量不要在`fields`处指定类型，而是在复合类型处指定，这样julia会自动convert你设置的`fields`值到复合类型指定的类型，更加简洁

In [46]:
# 不恰当的做法
@which Point3D(Int64(1), Int64(2), 3.0)

Point3D(x::T1, y::T1, z::T2) where {T1, T2} in Main at In[42]:2

### 参数化抽象类型

**restart kenerl**

In [47]:
abstract type Pointy{T} end

In [48]:
mutable struct Point1D{T} <: Pointy{T}
    x::T
end

LoadError: invalid redefinition of constant Point1D

In [49]:
mutable struct Poin2D{T} <: Pointy{T}
    x::T
    y::T
end

In [50]:
mutable struct Point3D{T} <: Pointy{T}
    x::T
    y::T
    z::T
end

LoadError: invalid redefinition of constant Point3D

In [51]:
function module_t(p::Pointy)
    m = 0
    for field in fieldnames(typeof(p))
        m += getfield(p, field)^2
    end
    return sqrt(m)
end

module_t (generic function with 1 method)

In [52]:
p1 = Poin2D{Float64}(2, 3.0)

Poin2D{Float64}(2.0, 3.0)

In [53]:
module_t(Poin2D(2.0, 3.0))

3.605551275463989

In [54]:
fieldnames(typeof(p1))[1]

:x

In [55]:
getfield(p1, :x)

2.0

### 参数化元类型

In [56]:
primitive type ptr{T} 64 end

### 参数化基本原理（跳过）

从这一块到最后有点麻烦，先跳过

+ `Union`是一个类型，`Union{T}`是`Union`的实例，`T`不为空时仍然是一个类型
+ `Type`是一个类型，`Type{T}`是`Type`的一个实例，仍然是一个类型
+ `DataType`是一个类型，也是一个实例
+ `Type{T}`是`Any`的子类型，是`DataType`,`UnionAll`,`Union`,`Core.TypeofBottom`的父类型

In [57]:
supertype(Union)

Type{T}

In [58]:
typeof(Union{})

Core.TypeofBottom

In [59]:
supertype(Union{Int64})

Signed

In [60]:
supertype(Any)

Any

In [61]:
typeof(Core.TypeofBottom)

DataType

In [62]:
typeof(Union{})

Core.TypeofBottom

In [63]:
typeof(Type{})

UnionAll

In [64]:
typeof(Type)

UnionAll

### 参数化继承(跳过）

## 常用数据类型

### 元组

In [65]:
tp1 = (1.0, 2.0, 3.0, 4.0)

(1.0, 2.0, 3.0, 4.0)

In [66]:
typeof(tp1)

NTuple{4, Float64}

In [67]:
# NTuple不是一个复合类型
tp3 = NTuple{3, Float64}(1.0, 2.0, 3.0)

LoadError: MethodError: no method matching Tuple{Float64, Float64, Float64}(::Float64, ::Float64, ::Float64)
[0mClosest candidates are:
[0m  (::Type{T})(::Any) where T<:Tuple at C:\Users\XJZ\AppData\Local\julias\julia-1.7\share\julia\base\tuple.jl:317

In [68]:
tp1[1]

1.0

In [69]:
tp2 = (1, 2, (3, 4, 5)..., 6, 7)

(1, 2, 3, 4, 5, 6, 7)

In [70]:
tp3 = (1, "a", "china")

(1, "a", "china")

### 命名元组

In [71]:
ntp1 = (a = 1, b = 2, c = 3)

(a = 1, b = 2, c = 3)

In [72]:
typeof(ntp1)

NamedTuple{(:a, :b, :c), Tuple{Int64, Int64, Int64}}

In [73]:
ntp1.a

1

In [74]:
# 说明命名元组是一个参数化复合类型
ntp2 = NamedTuple{(:a, :b, :c),Tuple{Int64,Int64,Float64}}((1, 2, 3.0))

(a = 1, b = 2, c = 3.0)

### 键值对

In [75]:
dump(Pair) # 先只关注body部分吧，其它的部分先别管

UnionAll
  var: TypeVar
    name: Symbol A
    lb: Union{}
    ub: Any
  body: UnionAll
    var: TypeVar
      name: Symbol B
      lb: Union{}
      ub: Any
    body: Pair{A, B} <: Any
      first::A
      second::B


In [76]:
fieldnames(Pair)

(:first, :second)

In [77]:
p1 = Pair(1, 3.2)

1 => 3.2

In [78]:
typeof(p1)

Pair{Int64, Float64}

In [79]:
p2 = 2 => "3"

2 => "3"

In [80]:
p1.first

1

In [81]:
p2.second

"3"

In [82]:
# Pair to Turple
Tuple(p1)

(1, 3.2)

### 字典

In [83]:
d1 = Dict(1 => 1.1, 2 => 2.2)

Dict{Int64, Float64} with 2 entries:
  2 => 2.2
  1 => 1.1

In [ ]:
d2 = Dict{String,Int64}("a" => 3, "b" => 4.0)

In [85]:
# 字典推导式
d3 = Dict(i => i^2 for i ∈ 1:10)

Dict{Int64, Int64} with 10 entries:
  5  => 25
  4  => 16
  6  => 36
  7  => 49
  2  => 4
  10 => 100
  9  => 81
  8  => 64
  3  => 9
  1  => 1

In [86]:
ntp2

(a = 1, b = 2, c = 3.0)

In [87]:
pairs(ntp2)

pairs(::NamedTuple) with 3 entries:
  :a => 1
  :b => 2
  :c => 3.0

In [88]:
# NamedTuple to Dict
d4 = Dict(pairs(ntp2))

Dict{Symbol, Real} with 3 entries:
  :a => 1
  :b => 2
  :c => 3.0

In [89]:
d4[:b]

2

In [90]:
delete!(d4, :a)

Dict{Symbol, Real} with 2 entries:
  :b => 2
  :c => 3.0

In [91]:
pop!(d4, :b)

2

In [92]:
d4

Dict{Symbol, Real} with 1 entry:
  :c => 3.0

In [93]:
d4.count

1

In [94]:
keys(d2)

KeySet for a Dict{String, Int64} with 2 entries. Keys:
  "b"
  "a"

In [95]:
for key in keys(d2)
    println("$key \t", d2[key])
end

b 	4
a 	3


In [96]:
values(d2)

ValueIterator for a Dict{String, Int64} with 2 entries. Values:
  4
  3

In [97]:
# collect iterator
collect(values(d2))

2-element Vector{Int64}:
 4
 3

In [98]:
# Set(可迭代数据结构)
a = Set(1:5)

Set{Int64} with 5 elements:
  5
  4
  2
  3
  1

In [99]:
b = Set(1, 2, 3)

LoadError: MethodError: no method matching Set(::Int64, ::Int64, ::Int64)
[0mClosest candidates are:
[0m  Set(::Any) at C:\Users\XJZ\AppData\Local\julias\julia-1.7\share\julia\base\set.jl:23

In [ ]:
b = Set(("a", 1, 2))

In [101]:
b[1]

LoadError: MethodError: no method matching getindex(::Set{Any}, ::Int64)

In [102]:
1 in b

true

In [103]:
isempty(a)

false

In [104]:
empty!(a)

Set{Int64}()

### 缺失值missing 

In [105]:
# NaN是一个数并不代表缺失值

In [106]:
missing + 1

missing

In [ ]:
ismissing([1, 2, 3, missing])

In [ ]:
ismissing.([1, 2, 3, missing])

In [109]:
isless(missing, Inf)

false

### nothing不需要在内存中表达

## 其它数据类型

Julia有一个`DataStructures.jl`， 这个包内置了很多Julia本身没有的数据结构

### `OrderedDict`

In [1]:
using DataStructures

In [16]:
# 有序字典
d1 = OrderedDict('a' => 1, 'b' => 2, 'c' => 3)

OrderedDict{Char, Int64} with 3 entries:
  'a' => 1
  'b' => 2
  'c' => 3

In [20]:
# 无序字典（Julia内置）
d2 = Dict('a' => 1, 'b' => 2, 'c' => 3)

Dict{Char, Int64} with 3 entries:
  'a' => 1
  'c' => 3
  'b' => 2

In [23]:
?methodswith

search: methodswith



```
methodswith(typ[, module or function]; supertypes::Bool=false])
```

Return an array of methods with an argument of type `typ`.

The optional second argument restricts the search to a particular module or function (the default is all top-level modules).

If keyword `supertypes` is `true`, also return arguments with a parent type of `typ`, excluding type `Any`.


`methodswith`函数可以查看所有以某个数据结构为参数的所有函数， 例如`methodswith(OrderedDict)`可以查看所有适用于`OrderedDict`的函数， 可以看出包的作者给`OrderedDict`适配了很多的方法

In [24]:
methodswith(OrderedDict)

[1] convert(::Type{OrderedDict{K, V}}, d::OrderedDict{K, V}) where {K, V} in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:110
[2] copy(d::OrderedDict) in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:50
[3] delete!(h::OrderedDict, key) in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:441
[4] empty(d::OrderedDict{K, V}) where {K, V} in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:79
[5] empty(d::OrderedDict, ::Type{K}, ::Type{V}) where {K, V} in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:80
[6] empty!(h::OrderedDict{K, V}) where {K, V} in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:220
[7] get(default::Union{Function, Type}, h::OrderedDict{K, V}, key) where {K, V} in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:388
[8] get(h::OrderedDict{K, V}, key, default) where {K, V} in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:383
[9] get!(default::Union{Function, Type}, h::OrderedDict{K, V}, key0) where {K, V} in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:354
[10] get!(h::OrderedDict{K, V}, key0, default) where {K, V} in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:339
[11] getindex(h::OrderedDict{K, V}, key) where {K, V} in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:378
[12] getkey(h::OrderedDict{K, V}, key, default) where {K, V} in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:396
[13] haskey(h::OrderedDict, key) in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:393
[14] isempty(d::OrderedDict) in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:83
[15] iterate(t::OrderedDict) in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:447
[16] iterate(t::OrderedDict, i) in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:452
[17] length(d::OrderedDict) in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:82
[18] merge(d::OrderedDict, others::AbstractDict...) in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:466
[19] merge(combine::Function, d::OrderedDict, others::AbstractDict...) in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:471
[20] pop!(h::OrderedDict) in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:407
[21] pop!(h::OrderedDict, key) in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:421
[22] pop!(h::OrderedDict, key, default) in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:426
[23] popfirst!(h::OrderedDict) in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:414
[24] setindex!(h::OrderedDict{K, V}, v0, key0) where {K, V} in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:320
[25] similar(d::OrderedDict) in OrderedCollections at deprecated.jl:70
[26] sizehint!(d::OrderedDict, newsz) in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\ordered_dict.jl:206
[27] sort(d::OrderedDict; args...) in OrderedCollections at C:\Users\XJZ\.julia\packages\OrderedCollections\PRayh\src\dict_sorting.jl:20
[28] sort!(d::OrderedDict; byvalue, args...)

In [31]:
zipit = zip(1:5, 'a':'e')

zip(1:5, 'a':1:'e')

In [38]:
OrderedDict(zipit)

OrderedDict{Int64, Char} with 5 entries:
  1 => 'a'
  2 => 'b'
  3 => 'c'
  4 => 'd'
  5 => 'e'